Demo for AI Practicum




Names: Quinn Liu, Olivia Lu, Eli Zhang

Netids: qjl3, ol48, ekz5



# Run this code every time to start: 

In [1]:
from google.colab import drive
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
import io
import torch
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import torchvision.transforms as transforms              # composable transforms
from IPython.display import HTML, Image
from google.colab.output import eval_js, register_callback
from base64 import b64decode

drive.mount('/content/drive', force_remount=True)

#REINITIALIZE THESE 
data_path = os.path.join(os.getcwd(), "drive", "My Drive", "CS 4701 Prac", "data", "quickdraw_simplified")
flattened_data_path = os.path.join(os.getcwd(), "drive", "My Drive", "CS 4701 Prac", "data")
# dataset = np.load(data_path, encoding='latin1', allow_pickle=True)


Mounted at /content/drive


In [2]:
paths = os.listdir(data_path)
classes = {}
for i, f in enumerate(paths):
    classes[f[:-7]] = i
classes_list = list(classes.keys())

#Helper Methods and Preprocessing



In [3]:
# Lambda to switch to GPU if available
get_device = lambda : "cuda:0" if torch.cuda.is_available() else "cpu"
print(torch.cuda.is_available())

True


In [4]:
class SketchNet(nn.Module):
    def __init__(self, num_classes=16, batch_size=1):
        super(SketchNet, self).__init__()
        #Define layers of model architecture
        #out channel = 3, kernel_size = 3, stride =2, padding = 1
        self.conv1 = nn.Conv2d(1, 6, 3, 2,1)
        self.bn1 = nn.BatchNorm2d(6)

        #out channel = 12, kernel_size = 3, stride =2, padding = 1
        self.conv2 = nn.Conv2d(6, 12, 3, 2, 1)
        self.bn2 = nn.BatchNorm2d(12)

        #out channel = 24, kernel_size = 3, stride =2, padding = 1
        self.conv3 = nn.Conv2d(12, 24, 3, 2, 1) #24x32x32

        self.bn3 = nn.BatchNorm2d(24)
        self.avgpool = nn.AvgPool2d(7, 1) #24x26x26
        #fc
        self.fc = nn.Linear(batch_size*24*26*26, 128)
        #cls 
        self.cls = nn.Linear(128, num_classes)

        self.relu = nn.LeakyReLU()
    def forward(self, x):
        x = x.contiguous().view(-1, 1, 255, 255).to(get_device())
        # Define forward pass
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        #fc
        x = self.fc(x)
        x = self.relu(x)
        x = self.cls(x)
        return x

In [5]:
def classify(ex, model, c):
  image = PIL.Image.open(io.BytesIO(ex)) 
  arr = np.asarray(image)
  img_gray = 255 - arr[:, :, 3]
  transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])
  output = model(transform(img_gray).to(get_device()))
  predicted = torch.max(output.data, 1)[1].to(get_device())
  lab = predicted.to("cpu")[0].item()
  return c[lab]

## Running the example

In [6]:
model = SketchNet(len(classes)).to(get_device())
model.load_state_dict(torch.load("model.pt"))

<All keys matched successfully>

In [7]:
# print(classes_list)
# val = input("Enter your image name: ") 

In [8]:

#this code was taken from github
#https://gist.github.com/korakot/8409b3feec20f159d8a50b0a811d3bca

canvas_html = """
<canvas width=%d height=%d></canvas>
<button id=0>Erase</button>
<button id1>Stop</button>
<script>
var canvas = document.querySelector('canvas')
var button = document.getElementById(0)
var stopButton = document.getElementById(1)
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
var mouse = {x: 0, y: 0}
var data;
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}
button.onclick = () => {
  ctx.clearRect(0, 0, canvas.width, canvas.height);
}
var running = setInterval(() => {
  data = canvas.toDataURL('image/png')
  google.colab.kernel.invokeFunction('notebook.displayPrediction', [], {});
}, 1000);
stopButton.onclick = () => {
  clearInterval(running);
}
</script>
"""

def display_prediction():
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(flattened_data_path + "/" + "current_drawing.png", 'wb') as f:
    f.write(binary)
  prediction = classify(binary, model, classes_list)
  print('\r', prediction, end='')

register_callback('notebook.displayPrediction', display_prediction)
display(HTML(canvas_html % (255, 255, 3)))

# def draw(filename='drawing', w=255, h=255, line_width=1):
#   display(HTML(canvas_html % (w, h, line_width)))
#   data = eval_js("data")
#   binary = b64decode(data.split(',')[1])
#   with open(flattened_data_path + "/" + filename +".png", 'wb') as f:
#     f.write(binary)
#   return binary

 flip flops

In [9]:
# fname = val
# example = draw(fname)

In [10]:
prediction = classify(example, model, classes_list)
print(prediction)

NameError: ignored